In [ ]:
import time
import json
import tweepy

In [ ]:
# Keys and Access Tokens
consumer_key = 'ZsdCp41WfGyjRhqGrXLijllaL'
consumer_secret = 'aQTeoGnNxctaLqj5gw5AqJ8liqFtAWedyqoZL6naQobOtYQGo0'

access_token = '848211984399179776-F0IxgJAanurK4e8lYzBjNhpzY8aENDw'
access_token_secret = 'SvJP296wgdOcNlHJGckCgRNtXEotPvYORP05TJOPomb0t'

In [ ]:
def get_twitter_api_access(consumer_key, consumer_secret, access_token, access_token_secret):
    """Set up Authentication Instance"""

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

In [ ]:
# The API class provides access to the entire twitter RESTful API methods.
api = get_twitter_api_access(consumer_key, consumer_secret, access_token, access_token_secret)

---

### 1. Explore Trending Topics with Location

In [ ]:
BRAZIL = 23424768 # parent id
SAO_PAULO = 455827 # id

EUA = 23424977
NEW_YORK = 2459115

# Reference:
# https://developer.twitter.com/en/docs/twitter-api/v1/trends/trends-for-location/api-reference/get-trends-place
tts = api.get_place_trends(SAO_PAULO)

# Find more locations with
# "GET https://api.twitter.com/1.1/trends/available.json"
# Try on Insomnia

print ('CURRENT TOP 50 TRENDING TOPICS\n')
for index, tt in enumerate(tts[0]['trends']):
    print (f"{index+1}. {tt['name']} (Volume: {tt['tweet_volume']})")

---

## Streaming

In [ ]:
class MyStreamListener(tweepy.Stream):
    """Listen for tweets"""
    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret, api):
        super(MyStreamListener, self).__init__(consumer_key, consumer_secret, access_token, access_token_secret)
        self.counter = 0
        # References the auth instance for the listener
        self.api = api    
    
    def on_connect(self):
        """
        This is called after successfully connecting to the streaming API.
        """
        print("Connected to Twitter API!")
        
    def on_connection_error(self):
        """
        This is called when the stream connection errors or times out.
        """
        print("Error connect to Twitter API!")
        
    def on_status(self, tweet):
        """
        This is called when a status is received.
        """         
        user = tweet.user.screen_name
        text = tweet.text
        date_created = tweet.created_at

        print (user, text, date_created)
        
    def on_data(self, raw_data):
        """
        This is called when raw data is received from the stream. 
        This method handles sending the data to other methods based 
        on the message type.
        
        Object Format: 
            - https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
        """
        try:
            tweet = json.loads(raw_data.decode('utf-8'))
            user = tweet['user']
            
            # TODO
            # Filter RTs
            
            print ('--' * 50)
            
            print ('\nTweet information:')
            print (f"  - Tweet ID: {tweet['id']}")
            print (f"  - Created at: {tweet['created_at']}")
            print (f"  - Text: {tweet['text']}")
            
            print ('\nUser information:')
            print (f"  - Name: {user['name']}")
            print (f"  - Username: @{user['screen_name']}")
            
            print ('\nMetadata:')
            print (tweet['entities'])
            
            print ('--' * 50)
                 
            
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
    
    def on_disconnect(self):
        """
        This is called when the stream has disconnected.
        """
        print("The connection to Twitter API has been closed!")
        
    def on_keep_alive(self):
        """
        This is called when a keep-alive signal is received.
        """
        print("Keep-Alive signal received!")
        

### 2. Retrieve Streaming Data (Keywords)

#### References

- https://docs.tweepy.org/en/stable/stream.html#tweepy.Stream.on_status
- https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [ ]:
# Streams utilize Streaming HTTP protocol to deliver data through 
# an open, streaming API connection.

# search_topics = ['$ABEV3', '$PETR4', '$VALE3', '$WEGE3', '$OIBR3', '$VIIA3']

# twitter_listener = MyStreamListener(consumer_key, 
#                                     consumer_secret, 
#                                     access_token, 
#                                     access_token_secret, 
#                                     api)

# twitter_listener.filter(track=search_topics,
#                         languages=['pt'],
#                         threaded=True)

---

### 3. Retrieve Streaming Data (Users)



In [16]:
usernames = ['arkoadvice', 'sunonoticias', 'InvestingBrasil', 'leiamoneytimes', 'valorinveste', 'infomoney', 'B3_Oficial', 'Fatosdomercado']
find_users = [api.search_users(user) for user in usernames]

user_ids = []

print ('Following...\n')
for users_found in find_users:
    for user in users_found:
        data = user._json

        if data['screen_name'] in usernames:
            print ('  ', data['name'], data['id'])
            #print (data['screen_name'])
            user_ids.append(data['id'])

Following...

   Arko Advice 1336365702698192896
   Suno Notícias 778276232081342464
   Investing.com Brasil 🇧🇷 49292227
   Money Times 51150679
   Valor Investe 1113533606247436289
   InfoMoney | IM+ 59773459
   B3 74550038
   Fatos Relevantes S.A. 1124834863520022529


In [ ]:
# The default access level allows up to 400 track keywords and 5,000 follow userids.

# twitter_listener = MyStreamListener(consumer_key, 
#                                     consumer_secret, 
#                                     access_token, 
#                                     access_token_secret, 
#                                     api)

# twitter_listener.filter(follow=user_ids,
#                         languages=['pt'],
#                         threaded=True)

---

### 4. Retrieve Tweet History of Specific User

In [12]:
def display_hashtag_info(hashtag_list):
    if len(hashtag_list) == 0:
        print ('\tNo Hashtags mentioned.')
    else:
        for index, hashtag in enumerate(hashtag_list):
            print (f"\t{index+1}. {hashtag['text'].upper()}")

In [13]:
def display_url_info(url_list):
    if len(url_list) == 0:
        print ('\tNo URLs mentioned.')
    else:
        for index, url in enumerate(url_list):
            print (f"\t{index+1}. {url['expanded_url']}")

In [14]:
def display_media_info(entities_json):
    if 'media' not in entities_json.keys():
        print ('\tNo media available.')
    else:
        media_list = entities_json['media']
        for index, media in enumerate(media_list):
            print (f"\t{index+1}. {media['type']} {media['url']}")

In [11]:
def display_tweet_info(tweet):
    # User Specific Info
    user = tweet['user']
    
    print ('--' * 50)
            
    print ('\nTweet information:')
    print (f"  - Tweet ID: {tweet['id']}")
    print (f"  - Created at: {tweet['created_at']}")
    print (f"  - Text: {tweet['text']}")

    print ('\nUser information:')
    print (f"  - User ID: {user['id']}")
    print (f"  - Name: {user['name']}")
    print (f"  - Username: @{user['screen_name']}")

    print ('\nMetadata:')
    print (f"  - Hashtags: ")
    display_hashtag_info(tweet['entities']['hashtags'])
    print (f"  - URLs: ")
    display_url_info(tweet['entities']['urls'])
    print (f"  - Media: ")
    display_media_info(tweet['entities'])
    
    #print (tweet['entities'])

In [9]:
# This method can only return up to 3,200 of a user's most recent Tweets.
user_timeline = api.user_timeline(user_id=1124834863520022529,
                                 screen_name='Fatosdomercado',
                                 count=5,
                                 exclude_replies=True, 
                                 include_rts=False)

In [15]:
for t in user_timeline:
    tweet = t._json
    display_tweet_info(tweet)
    print ('\n')

----------------------------------------------------------------------------------------------------

Tweet information:
  - Tweet ID: 1520033166182035456
  - Created at: Fri Apr 29 13:32:02 +0000 2022
  - Text: As corretoras deveriam demitir todos os analistas. Além de errarem tudo, ainda levam os investidores juntos.
Onde s… https://t.co/6TgCsZQYbO

User information:
  - User ID: 1124834863520022529
  - Name: Fatos Relevantes S.A.
  - Username: @Fatosdomercado

Metadata:
  - Hashtags: 
	No Hashtags mentioned.
  - URLs: 
	1. https://twitter.com/i/web/status/1520033166182035456
  - Media: 
	No media available.




---

### 5. Get information of Specific Tweet ID

In [ ]:
tweet_info = api.get_status(1492976710215741446)

In [ ]:
tweet_info._json

---

### 6. Store Collected Data

In [ ]:
def proccess_user_timeline(user_timeline):
    
    results_list = list()
    
    for t in user_timeline:
        
        tweet = t._json
        
        results_dict = dict()
        
        results_dict['tweet_id'] = ref_date
        results_dict['user_id'] = ref_date
        results_dict['created_at'] = ref_date
        results_dict['tweet_text'] = ref_date
        
        results_dict['beta_ticker2'] = ref_date
        results_dict['\"window\"'] = ref_date
        results_dict['correlation'] = ref_date
        results_dict['daily_vol_pair'] = ref_date

        # Save Results
        results_list.append(results_dict)
        
    return results_list